# Création de prompts pour le nommage et la description des clusters

In [1]:
Sys.setlocale("LC_ALL", "fr_CA.UTF-8")

# Library
library(dplyr)
library(tidyr)
library(ggplot2)
library(ggtext)
library(tidytext)
library(ellipsellm)

# Load Data and Clustering Results
kmeans_result <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/03_pilot1_kmeans.rds") # nolint
dfUsedForClustering <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/03_pilot1_2022.rds")
df_clusters_composition <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/03_pilot1_clusters_composition.rds")

[1] "fr_CA.UTF-8/fr_CA.UTF-8/fr_CA.UTF-8/C/fr_CA.UTF-8/C"


Attachement du package : ‘dplyr’


Les objets suivants sont masqués depuis ‘package:stats’:

    filter, lag


Les objets suivants sont masqués depuis ‘package:base’:

    intersect, setdiff, setequal, union




In [ ]:
cluster_names <- unique(dfUsedForClustering$cluster_name)
cluster_names <- cluster_names[order(cluster_names)]

prompts <- list()

for (cluster_value in cluster_names) {
  cluster_data <- dfUsedForClustering %>% filter(cluster_name == cluster_value)
  cluster_vars <- df_clusters_composition %>% filter(cluster_name == cluster_value)

  # Calculer quelques statistiques démographiques pour le cluster
  perc_femmes     <- mean(cluster_data$female, na.rm = TRUE) * 100
  perc_immigrants <- mean(cluster_data$immigrant, na.rm = TRUE) * 100
  perc_langEn     <- mean(cluster_data$langEn, na.rm = TRUE) * 100
  perc_langFr     <- mean(cluster_data$langFr, na.rm = TRUE) * 100
  perc_ses_languageOther <- mean(cluster_data$ses_languageOther, na.rm = TRUE) * 100
  perc_ethn_white <- mean(cluster_data$ses_ethn_White, na.rm = TRUE) * 100
  perc_hetero     <- mean(cluster_data$ses_hetero, na.rm = TRUE) * 100
  educ_mean       <- mean(cluster_data$educ, na.rm = TRUE)
  age_mean        <- mean(cluster_data$age, na.rm = TRUE) * 80
  revenu_moyen    <- mean(cluster_data$ses_income, na.rm = TRUE) * 225000

  # Génération du prompt de suggestion de noms
  prompt_description <- (paste0(
    "Propose un prénom pour une personne habitant au Québec et donne une brève description pour le persona ", cluster_value, " :\n",

    "Variables avec une moyenne significativement plus élevée que la moyenne de la population en général :\n",
    if (any(cluster_vars$color == "green")) {
      paste(cluster_vars[cluster_vars$color == "green", "variable"], collapse = "\n")
    } else {
      "Aucune variable n'a une moyenne significativement plus élevée que la moyenne de la population en général."
    },
    "\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\n",
    if (any(cluster_vars$color == "red")) {
      paste(cluster_vars[cluster_vars$color == "red", "variable"], collapse = "\n")
    } else {
      "Aucune variable avec une moyenne significativement plus basse que la moyenne de la population en général."
    },
    "\n\nCe cluster est composé de \n", 
    "Femmes: ", round(perc_femmes, 1), "%, 
     Immigrants: ", round(perc_immigrants, 1), "%\n",
    "Âge: ", round(age_mean, 1), " ans, 
     Revenu: ", round(revenu_moyen, 1), " k$\n",
    "Langues: 
     En ", round(perc_langEn, 1), "%, 
     Fr ", round(perc_langFr, 1), "%, 
     Autres: ", round(perc_ses_languageOther, 1), "%\n",
    "Édu: ", round(educ_mean, 1), " (0=bas, 1=élevé), 
     Ethnie (Blanc): ", round(perc_ethn_white, 1), "%\n",
    "Orientation (Hétéro): ", round(perc_hetero, 1), "%\n"
  ))

  # Stocker le prompt dans la liste
  prompts[[cluster_value]] <- prompt_description
}

In [3]:
prompts

$`1`
[1] "Propose un prénom et donne une brève description pour le persona 1 :\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nmale\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nfemale\n\nCe cluster est composé de \nFemmes: 0.4%, \n     Immigrants: 6%\nÂge: 52.5 ans, \n     Revenu: 93714.8 k$\nLangues: \n     En 12.9%, \n     Fr 87.1%, \n     Autres: 0%\nÉdu: 0.5 (0=bas, 1=élevé), \n     Ethnie (Blanc): 98.3%\nOrientation (Hétéro): 100%\n"

$`2`
[1] "Propose un prénom et donne une brève description pour le persona 2 :\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"act_VisitsMuseumsGaleries\", \"ses_bisex\", \"ses_gai\")\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"age\", \"ses_hetero\")\n\nCe cluster est composé de \nFemmes: 38.4%, \n     Immigrants: 15.1%\nÂge: 36.2 ans, \n     Revenu: 79916.8 k$\nLangues: \n     En 17.1%, \n     Fr 82.9%, \n     Autres: 0%\nÉdu: 0.6 (0=bas, 1=élevé), \n     Ethnie (Blanc): 89%\nOrientation (Hétéro): 0%\n"

$`3`
[1] "Propose un prénom et donne une brève description pour le persona 3 :\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"act_VisitsMuseumsGaleries\", \"educ\", \"immigrant\")\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"age\", \"langFr\", \"ses_ethn_White\")\n\nCe cluster est composé de \nFemmes: 46.7%, \n     Immigrants: 74.8%\nÂge: 29.3 ans, \n     Revenu: 77619 k$\nLangues: \n     En 35.6%, \n     Fr 37.8%, \n     Autres: 26.7%\nÉdu: 0.7 (0=bas, 1=élevé), \n     Ethnie (Blanc): 12.6%\nOrientation (Hétéro): 94.1%\n"

$`4`
[1] "Propose un prénom et donne une brève description pour le persona 4 :\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nfemale\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nmale\n\nCe cluster est composé de \nFemmes: 99.7%, \n     Immigrants: 5.2%\nÂge: 47.9 ans, \n     Revenu: 81669.1 k$\nLangues: \n     En 10.3%, \n     Fr 89.7%, \n     Autres: 0%\nÉdu: 0.5 (0=bas, 1=élevé), \n     Ethnie (Blanc): 99%\nOrientation (Hétéro): 100%\n"

In [4]:
reponses_openai <- list()
# Afficher ou traiter tous les prompts
for (cluster in names(prompts)) {
  # Par exemple, envoyer chaque prompt à OpenAI (si la fonction est correctement définie)
  conv <- openai_create_conversation(prompts[[cluster]])
  resp <- openai_chat_completion(conv)
  reponses_openai[[cluster]] <- resp$content
}


In [5]:
reponses_openai

$`1`
[1] "Pour ce persona, je propose le prénom \"Philippe\".\n\nVoici une brève description de Philippe :\n\nPhilippe est un homme de 52 ans, d'origine française, vivant confortablement avec un revenu annuel de 93,714.8 k$. Il a un niveau d'éducation relativement élevé, ce qui lui permet de bien s'intégrer dans son environnement professionnel et social. Bien que sa communauté soit très homogène sur le plan ethnique (98.3% Blanc), Philippe entretient des relations amicales avec des personnes de diverses origines, y compris des immigrants qui représentent 6% du groupe.\n\nPhilippe est un francophone, mais il parle aussi couramment l'anglais, ce qui lui permet de communiquer aisément dans des contextes internationaux ou multiculturels. Bien qu'il soit plutôt à l'aise dans ses habitudes et son cercle social, il reste ouvert d'esprit et est souvent intéressé par les cultures et les perspectives différentes.\n\nSur le plan personnel, Philippe est hétérosexuel et apprécie les valeurs familiales traditionnelles, ce qui le pousse souvent à privilégier les relations stables et harmonieuses. Dans ses loisirs, il aime les activités qui lui permettent de se détendre et de profiter de la vie, que ce soit la lecture, les voyages ou les rencontres avec ses amis."

$`2`
[1] "Pour ce persona, je propose le prénom \"Alexis\".\n\n**Description d'Alexis :**\n\nAlexis est une personne jeune, âgée d'environ 36 ans, vivant dans une métropole francophone. Son revenu annuel est relativement élevé, ce qui lui permet de profiter d'activités culturelles telles que les visites de musées et galeries, qu'elle apprécie particulièrement. \n\nAlexis est ouvertement bisexuelle ou gaie, ce qui est reflété par le fait que la moyenne de la variable \"ses_bisex\" et \"ses_gai\" est significativement plus élevée que celle de la population générale. Sa perspective sur l'orientation sexuelle est inclusive puisqu'aucune personne de ce groupe ne s'identifie comme hétérosexuelle.\n\nÉduquée et d'origine majoritairement blanche, Alexis valorise la diversité et l'inclusivité, vivant dans un milieu où l'immigration représente une petite mais significative proportion (15.1%). Elle parle couramment le français, mais elle a aussi une compréhension de l'anglais, bien que ce soit sa langue secondaire.\n\nElle est engagée dans la communauté LGBTQ+ et participe activement à des événements qui célèbrent cette diversité. Sa curiosité intellectuelle et son amour pour l'art et la culture la poussent à être toujours à la recherche de nouvelles expériences et connaissances."

$`3`
[1] "Pour ce persona, je propose le prénom \"Lina\". Voici une brève description :\n\n**Lina** est une jeune femme de 29 ans, qui s'investit activement dans la découverte et l'appréciation de la culture, notamment par ses fréquentes visites dans les musées et galeries. Elle est très instruite, avec un niveau d'éducation supérieur à la moyenne. Lina est immigrante, faisant partie des 74,8% de ce cluster qui sont d'origine étrangère. Elle se distingue également par son jeune âge, bien en dessous de la moyenne de la population générale.\n\nBien qu'elle soit multilingue, parlant couramment le français et l'anglais, elle communique aussi dans d'autres langues. Lina vit dans un milieu multiculturel et ne s'identifie pas comme blanche, ce qui est en accord avec le faible pourcentage de son ethnie dans le cluster. Son revenu est relativement confortable, permettant un style de vie aisé. Elle est majoritairement hétérosexuelle, reflétant la norme de son groupe.\n\nLina est une personne ouverte sur le monde, appréciant les échanges culturels et les nouvelles expériences, ce qui enrichit sa perspective globale et nourrit sa passion pour les arts et la culture."

$`4`
[1] "Prénom proposé : Claire\n\nDescription : Claire est une femme de 47 ans, vivant principalement en France. Elle est d’origine blanche et parle couramment le français. Claire a un revenu confortable de 81 669 € par an. Bien qu'elle ait un niveau d'éducati

In [6]:
description_clusters <- data.frame(prompt = unlist(prompts), description = unlist(reponses_openai))

In [7]:
description_clusters

,prompt,description
,<chr>,<chr>
1,"Propose un prénom et donne une brève description pour le persona 1 : Variables avec une moyenne significativement plus élevée que la moyenne de la population en général : male Variables avec une moyenne significativement plus basse que la moyenne de la population en général : female Ce cluster est composé de Femmes: 0.4%, Immigrants: 6% Âge: 52.5 ans, Revenu: 93714.8 k$ Langues: En 12.9%, Fr 87.1%, Autres: 0% Édu: 0.5 (0=bas, 1=élevé), Ethnie (Blanc): 98.3% Orientation (Hétéro): 100%","Pour ce persona, je propose le prénom ""Philippe"". Voici une brève description de Philippe : Philippe est un homme de 52 ans, d'origine française, vivant confortablement avec un revenu annuel de 93,714.8 k$. Il a un niveau d'éducation relativement élevé, ce qui lui permet de bien s'intégrer dans son environnement professionnel et social. Bien que sa communauté soit très homogène sur le plan ethnique (98.3% Blanc), Philippe entretient des relations amicales avec des personnes de diverses origines, y compris des immigrants qui représentent 6% du groupe. Philippe est un francophone, mais il parle aussi couramment l'anglais, ce qui lui permet de communiquer aisément dans des contextes internationaux ou multiculturels. Bien qu'il soit plutôt à l'aise dans ses habitudes et son cercle social, il reste ouvert d'esprit et est souvent intéressé par les cultures et les perspectives différentes. Sur le plan personnel, Philippe est hétérosexuel et apprécie les valeurs familiales traditionnelles, ce qui le pousse souvent à privilégier les relations stables et harmonieuses. Dans ses loisirs, il aime les activités qui lui permettent de se détendre et de profiter de la vie, que ce soit la lecture, les voyages ou les rencontres avec ses amis."
2,"Propose un prénom et donne une brève description pour le persona 2 : Variables avec une moyenne significativement plus élevée que la moyenne de la population en général : c(""act_VisitsMuseumsGaleries"", ""ses_bisex"", ""ses_gai"") Variables avec une moyenne significativement plus basse que la moyenne de la population en général : c(""age"", ""ses_hetero"") Ce cluster est composé de Femmes: 38.4%, Immigrants: 15.1% Âge: 36.2 ans, Revenu: 79916.8 k$ Langues: En 17.1%, Fr 82.9%, Autres: 0% Édu: 0.6 (0=bas, 1=élevé), Ethnie (Blanc): 89% Orientation (Hétéro): 0%","Pour ce persona, je propose le prénom ""Alexis"". **Description d'Alexis :** Alexis est une personne jeune, âgée d'environ 36 ans, vivant dans une métropole francophone. Son revenu annuel est relativement élevé, ce qui lui permet de profiter d'activités culturelles telles que les visites de musées et galeries, qu'elle apprécie particulièrement. Alexis est ouvertement bisexuelle ou gaie, ce qui est reflété par le fait que la moyenne de la variable ""ses_bisex"" et ""ses_gai"" est significativement plus élevée que celle de la population générale. Sa perspective sur l'orientation sexuelle est inclusive puisqu'aucune personne de ce groupe ne s'identifie comme hétérosexuelle. Éduquée et d'origine majoritairement blanche, Alexis valorise la diversité et l'inclusivité, vivant dans un milieu où l'immigration représente une petite mais significative proportion (15.1%). Elle parle couramment le français, mais elle a aussi une compréhension de l'anglais, bien que ce soit sa langue secondaire. Elle est engagée dans la communauté LGBTQ+ et participe activement à des événements qui célèbrent cette diversité. Sa curiosité intellectuelle et son amour pour l'art et la culture la poussent à être toujours à la recherche de nouvelles expériences et connaissances."
3,"Propose un prénom et donne une brève description pour le persona 3 : Variables avec une moyenne significativement plus élevée que la moyenne de la population en général : c(""act_VisitsMuseumsGaleries"", ""educ"", ""immigrant"") Variables avec une moyenne significativement plus basse que la moyenne de la population en général : c(""age"", ""langFr"", ""ses_ethn_White"") Ce cluster est 

In [8]:
saveRDS(description_clusters, file = "../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/03_pilot1_clusters_description.rds")